<a href="https://colab.research.google.com/github/czelijm/SNR_18Z/blob/master/SNR_Zadanie3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Podpinanie dysku google, importowanie bibliotek oraz definicje funkcji


In [0]:
!pip install -q keras


##Uzyskanie autoryzacji do korzystania z chmury google

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

##ustalanie folderu


In [0]:
cd /content/drive/snr_projekt

/content/drive/snr_projekt


## Import bibliotek, definicje funkcji

In [0]:
import os
from collections import Counter

import cv2
import matplotlib.pyplot as plt
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from skimage import feature
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

from keras.applications.vgg16 import VGG16

from keras import layers
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation, InputLayer
from matplotlib import pyplot
from skimage.util import random_noise

from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn import svm

import seaborn as sns
import sys


In [0]:
def check_numpy_set_exist(name):
    if os.path.isfile("dataX-" + name + ".npy") and os.path.isfile("dataY-" + name + ".npy"):
        return True
    return False

In [0]:
def load_numpy_set(name):
    x = np.load("dataX-" + name + ".npy")
    y = np.load("dataY-" + name + ".npy")
    return x, y

In [0]:
def make_id_class_dictionary(y_val):
    # make dictionary of classes
    y_val.sort()
    y_val = Counter(y_val).keys()

    class_id = {}
    temp_list_id = []
    count = 1

    for i in y_val:
        temp_list_id.append((i, count))
        count += 1

    for word, _id in temp_list_id:
        class_id[word] = _id
    print(class_id)

    return class_id

In [0]:
def cvt2_id_class_list(dictionary_class_id, word_list):
    # covert word list of class to id list of class
    y_teach = []
    for i in word_list:
        y_teach.append(dictionary_class_id[i])
    return y_teach

In [0]:
def cv2_id_class_as_vector(y_val_temp, num_of_class):
    y_val = []
    for i in y_val_temp:
        y = [0] * num_of_class
        y[i - 1] = 1
        y_val.append(y)
    np.asarray(y_val)
    return y_val

In [0]:
def to_labels(y):
  return np.argmax(y,1)

In [0]:
def compare_results(y_pred_lab, y_test_lab):
  acc = np.mean(y_pred_lab==y_test_lab)
  return acc

#Zadanie 3 porównanie kolejnych   rzędów wielomianów  (począwszy od liniowego do    5 stopnia) 

##Wczytywanie danych

In [0]:
print("Dane treningowe wczytywanie rozpoczęte")
x_teach, y_teach = load_numpy_set("teach")
print("Dane treningowe wczytane")


In [0]:
print("Dane testowe wczytywanie rozpoczęte")
x_test_temp, y_test_temp = load_numpy_set("test")
print("Dane testowe wczytane")
y_test_temp_copy = y_test_temp.copy()

In [0]:
# random Split Data set to teach
x_teach, x_val, y_teach_temp1, y_val_temp1 = train_test_split(x_teach, y_teach, test_size=0.3, random_state=42)
print("Dane podzielone na zbiór treningowy i walidacyjny")

In [0]:
# number of class in set
num_of_class = len(set(y_test_temp_copy))

# make idClasses
id_class_dictionary = make_id_class_dictionary(y_test_temp_copy)

# convert from wordList to numberList Of class
y_teach_temp2 = cvt2_id_class_list(id_class_dictionary, y_teach_temp1)
y_val_temp2 = cvt2_id_class_list(id_class_dictionary, y_val_temp1)

In [0]:
# get final representation of labels as vector ex. [0 0 0 0 ..... 0 1 0 0 0] - outputs of network
y_val = cv2_id_class_as_vector(y_val_temp2, num_of_class)
y_teach = cv2_id_class_as_vector(y_teach_temp2, num_of_class)

##wczytywanie modelu referencyjnego oraz wyznaczanie jego dokładności

In [0]:
y_train_lab = np.array(y_teach_temp2)
y_test_lab = np.array(y_val_temp2)

In [0]:
model_ref =load_model("best_VGG_based_model_sp.hdf5") #load_model("early_stopping_delta007.hdf5") #

In [0]:
y_pred = model_ref.predict(x_val)

In [0]:
model_ref.summary()

In [0]:
ref_te_acc, ref_te_cm = compare_results(to_labels(y_pred),y_test_lab)

In [0]:
ref_te_acc

In [0]:
y_pred = model_ref.predict(x_teach)

In [0]:
ref_tr_acc, ref_tr_cm = compare_results(to_labels(y_pred),y_train_lab)

In [0]:
ref_tr_acc

In [0]:
print_heatmap(ref_tr_cm)

##SVM OneVsOne

###Wczytywanie modelu sieci dla SVM

In [0]:
model = load_model("best_VGG_based_model_sp.hdf5") #load_model("early_stopping_delta007.hdf5")  #load_model("best_VGG_based_model_sp.hdf5")

In [0]:
model.summary()

In [0]:
X_train_out = model.predict(x_teach)

In [0]:
X_test_out = model.predict(x_val)

###SVM wielomian stopnia 1go

In [0]:
svm1 = svm.SVC(kernel='poly', degree=1, decision_function_shape='ovo')

In [0]:
svm1.fit(X_train_out,y_train_lab)

In [0]:
y_pred_lab = svm1.predict(X_test_out)

In [0]:
y_pred_lab.shape, y_test_lab.shape

In [0]:
accuracy = compare_results(y_pred_lab,y_test_lab)

In [0]:
accuracy

###SVM wszystkie stopnie wielomianów


In [0]:
te_accs = []
tr_accs = []
degrees = [1,2,3,4,5]
names = ['ref']
for degree in degrees:
  names.append('SVM dim='+str(degree))

In [0]:
te_accs.append(ref_te_acc)
tr_accs.append(ref_tr_acc)

In [0]:
names

['ref', 'SVM dim=1', 'SVM dim=2', 'SVM dim=3', 'SVM dim=4', 'SVM dim=5']

In [0]:
svms = []

In [0]:
#Tworzenie modeli
for degree in degrees:
  model = svm.SVC(kernel='poly', degree=degree, decision_function_shape='ovo')
  svms.append(model)

####wyszkolenie modeli

In [0]:
for idx,model in enumerate(svms):
  print("Training model "+str(idx)+"...")
  sys.stdout.flush()
  model.fit(X_train_out,y_train_lab)

In [0]:
np.save("all.npy",svms)

In [0]:
for idx,model in enumerate(svms):
  print("Testing model "+str(idx)+"...")
  sys.stdout.flush()
  #train
  y_pred_lab = model.predict(X_train_out)
  acc = compare_results(y_pred_lab,y_train_lab)
  tr_accs.append(acc)
  print("Acc train: "+str(acc))
  sys.stdout.flush()
  #test
  y_pred_lab = model.predict(X_test_out)
  acc = compare_results(y_pred_lab,y_test_lab)
  te_accs.append(acc)
  print("Acc test: "+str(acc))
  sys.stdout.flush()

In [0]:
np.save("all_tr_accs_ovo.npy",tr_accs)
np.save("all_te_accs_ovo.npy",te_accs)

####Wizualizacja wyników

In [0]:
sns.barplot(x = names, y = te_accs)
plt.title("Test accuracy")

In [0]:
sns.barplot(x = names, y = tr_accs)
plt.title("Train accuracy")

##SVM 'OneVsRest'

In [0]:
for model in svms:
  del model
del svms

In [0]:
model_ref = load_model("best_VGG_based_model_sp.hdf5")

In [0]:
y_pred = model_ref.predict(x_val)

In [0]:
ref_te_acc = compare_results(to_labels(y_pred),y_test_lab)

In [0]:
y_pred = model_ref.predict(x_teach)

In [0]:
ref_tr_acc = compare_results(to_labels(y_pred),y_train_lab)

###Siec dla SVM

In [0]:
model_1 = load_model("best_VGG_based_model_sp.hdf5")

In [0]:
model = Sequential()

model.add(layers.InputLayer((100, 100, 3)))
model.add(layers.Convolution2D(64, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block1_conv1'))
model.add( layers.Convolution2D(64, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block1_conv2'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add( layers.Convolution2D(128, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block2_conv1'))
model.add( layers.Convolution2D(128, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block2_conv2'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv1'))
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv2'))
model.add( layers.Convolution2D(256, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block3_conv3'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block 4
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv1'))
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv2'))
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block4_conv3'))
model.add(layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block 5
model.add(layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block5_conv1'))
model.add(layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block5_conv2'))
model.add( layers.Convolution2D(512, (3, 3),
                         activation='relu',
                         padding='same',
                         name='block5_conv3'))
model.add( layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

model.add(layers.Flatten())
# tmp_model_working.add(Dropout(0.3))
model.add(layers.Dense(71, activation='softmax'))


model.set_weights(model_1.get_weights()) 

In [0]:
model.summary()

In [0]:
model.pop()
model.pop()
model.summary()

In [0]:
X_train_out = model.predict(x_teach)

In [0]:
X_test_out = model.predict(x_val)

###SVM wszystkie stopnie wielomianów

In [0]:
te_accs = []
tr_accs = []
degrees = [1,2,3,4,5]
names = ['ref']
for degree in degrees:
  names.append('SVM dim='+str(degree))

In [0]:
te_accs.append(ref_te_acc)
tr_accs.append(ref_tr_acc)

In [0]:
names

['ref', 'SVM dim=1', 'SVM dim=2', 'SVM dim=3', 'SVM dim=4', 'SVM dim=5']

####usunięcie modelu

In [0]:
del model_ref
del model
del model_1

#### Utworzenie modeli

In [0]:
svms = []

In [0]:
#Tworzenie modeli
for degree in degrees:
  model = svm.SVC(kernel='poly', degree=degree, decision_function_shape='ovr')
  svms.append(model)

####szkolenie

In [0]:
for idx,model in enumerate(svms):
  print("Training model "+str(idx)+"...")
  sys.stdout.flush()
  model.fit(X_train_out,y_train_lab)

####Testowanie modeli

In [0]:
for idx,model in enumerate(svms):
  print("Testing model "+str(idx)+"...")
  sys.stdout.flush()
  #train
  y_pred_lab = model.predict(X_train_out)
  acc = compare_results(y_pred_lab,y_train_lab)
  tr_accs.append(acc)
  print("Acc train: "+str(acc))
  sys.stdout.flush()
  #test
  y_pred_lab = model.predict(X_test_out)
  acc = compare_results(y_pred_lab,y_test_lab)
  te_accs.append(acc)
  print("Acc test: "+str(acc))
  sys.stdout.flush()

In [0]:
np.save("all_tr_accs_ovac.npy",tr_accs)
np.save("all_tr_accs_ovac.npy",te_accs)

####Wizualizacja wyników

In [0]:

sns.barplot(x = names, y = te_accs)


In [0]:
sns.barplot(x = names, y = tr_accs)

In [0]:
te_accs